all the imports

In [7]:
import arxiv
import requests
import json

In [8]:
def arxiv_papers(query: str, max_results: int = 5) -> list[dict]:
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results = max_results)
    papers = []

    for result in client.results(search):
        paper = {
            "title" : result.title,
            "author": [result.author.name],
            "summary": result.summary,
            "published": result.published.date(),
            "url": result.pdf_url
        }
        papers.append(paper)

    return papers

In [10]:
def fetch_wiki(query:str, api_key:str, max_results:int = 5) -> list[dict]:
    url = "https://ieeexploreapi.ieee.org/api/v1/search/articles?parameter&apikey="
    params = {
        "query": query,
        'apikey': api_key,
        'max_records': max_results,
        'format': 'json'
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f'IEEE Xplore API failed: {response.text}')
    
    papers = []

    for article in response.json().get('articles', []):
        papers.append({
            "title": author.get('title', 'no title'),
            "authors": [author["full_name"] for author in article.get('authors', [])],
            "abstract":article.get('abstract', 'No abstract'),
            'published': article.get('publication_date',""),
            'url': article.get('pdf_url', article.get('html_url',''))
        })
    return papers

    